In [5]:
"""This is essentially the same as my denero non-61a review script applied to any 
ratemyprofessors page and any course filter (ex: sahai's page -> 189 reviews,
or sahai's page -> non 189 and non 16a reviews)"""
#import requests 
from bs4 import BeautifulSoup as bs
import re

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
PATH = r'C:\Users\jsonn\Documents\Python\selenium\chromedriver.exe' #needed to put r to not get unicode error, this line is from that youtube tutorial
#the path is where I put the chromedriver app thingie
grading_scale = {4.0:'A',
                    3.7: 'A-',
                    3.3: 'B+',
                     3.0: 'B',
                    2.7: 'B-',
                    2.3: 'C+',
                    2: 'C',
                    1.7:'C-',
                    1.3:'D+',
                     1:'D',
                     0.7:'D-',
                    0:'F'}
reversed_scale = {letter:num for num, letter in grading_scale.items()}

def create_driver(headless=True, implicit_wait=2):
    """
    opts = webdriver.chrome.options.Options()
    opts.headless = headless
    driver = webdriver.Remote(
        command_executor='http://127.0.0.1:3928/wd/hub',
        desired_capabilities=DesiredCapabilities.CHROME, 
        options=opts)
    driver.implicitly_wait(2)
    """
    opts = webdriver.chrome.options.Options()# changed from firefox to chrome 
    opts.headless = headless
    #driver = webdriver.Chrome(options=opts)
    driver = webdriver.Chrome(PATH)
    driver.implicitly_wait(implicit_wait)
    return driver

def average_grade(grades):
    """given a list of grades as numbers (i.e. A is a 4) returns the average grade as a letter.
    This function rounds down.
    average_grade([4.3,2.3,4,3.7,3.3]), where the mean is 3.52, rounds up to an A-
    average_grade([4,3,3,4,4,2.7]), where the mean is 3.449, rounds down to a b+"""
    number_grade = round(sum(grades) / len(grades), 1)
    for grade in grading_scale:
        if number_grade >= grade -.2: #starts with a, so if the average grade is >= 3.8
            return grading_scale[grade]
def grade_to_number(letter):
    if letter == "A+": #this is not really right but it's fine, there's no difference between the two
        letter = "A"
    return reversed_scale[letter]

In [6]:
url = input("paste link: ")
driver = create_driver(headless=False)
driver.get(url)
# Click cookie button
button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "FullPageModal__StyledCloseButton-sc-17feuxe-1")))
driver.execute_script("arguments[0].click();", button)

try:
    while True:
        button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, "Buttons__BlackButton-sc-19xdot-1")))
        driver.execute_script("arguments[0].click();", button)
except:
    pass

professor_soup = bs(driver.page_source, "html.parser")
driver.quit()

paste link: https://www.ratemyprofessors.com/ShowRatings.jsp?tid=1621181


In [58]:
filter_type = input("Choose the kind of filter you want. If you want a specific class(es), enter course number(s) (i.e. CS61A) one at a time, and enter 'done' when complete. If you want to filter OUT a course(s), just press enter for now: ")
if filter_type: #they need to put in the whole course name, and potentially other names that it might go by
    courses_wanted = [filter_type.upper()]
    while filter_type != "done":
        filter_type = input("choose another course. Enter 'done' when complete: ")
        courses_wanted.append(filter_type.upper())
    courses_wanted.pop() #gets rid of done, this doesn't really matter unless there's actually a course called done
print('Execute next cell.')

Choose the kind of filter you want. If you want a specific class(es), enter course number(s) (i.e. CS61A) one at a time, and enter 'done' when complete. If you want to filter OUT a course(s), just press enter for now: cs61a
choose another course. Enter 'done' when complete: done
Execute next cell.


In [59]:
course_names = professor_soup.find_all("div","RatingHeader__StyledClass-sc-1dlkqw1-2 hBbYdP")
names = [name.string for name in course_names][::2]
course_reviews = professor_soup.find_all("div", "Comments__StyledComments-dzzyvm-0 dEfjGB")
reviews = [review.string for review in course_reviews][1:] #we don't want most helpful rating because it's repeated 

#print(len(names), len(reviews)) i need to figure out why reviews is half the length of names 
#the length of names is resolved. the source code for some reason had stuff listed twice

course_ratings_difficulty = professor_soup.find_all("div", re.compile("RatingValues__RatingValue-sc-6dc747-3"))
#ratingvalues with 747-3 are course rating and difficulty
ratings, difficulties = [rating.string for rating in course_ratings_difficulty[::2]], [difficulty.string for difficulty in course_ratings_difficulty[1::2]]
print("Execute next cell.")

Execute next cell.


In [60]:
"""just getting more data in this cell. Would like to get course grades, and would take again.
Need to find out how to account for information that's not included to prevent inconsistency"""
#(professor_soup.find_all("div","MetaItem__StyledMetaItem-y0ixml-0 bxqwYh"))

"just getting more data in this cell. Would like to get course grades, and would take again.\nNeed to find out how to account for information that's not included to prevent inconsistency"

In [63]:
average_rating, average_difficulty, index, ratings_list, difficulties_list = 0, 0, 0, [], [] #by course wanted for if filter_type true
ratings_count = {x:0 for x in range(1,6)}
difficulty_count = {x:0 for x in range(1,6)}

if filter_type: #because the other option was "" #some implicit booleanness
    for name, review, rating, difficulty in zip(names, reviews, ratings, difficulties):
        print(name)
        if any([name in course for course in courses_wanted]):
     #   if name in courses_wanted:# or name[len(name)-3:] in [course[len(course)-3:] for course in courses_wanted]:# or name[len(name)-2:] in [course[len(course)-2:] for course in courses_wanted]:
            print(review, rating, difficulty, '\n')
            ratings_list.append(rating)
            difficulties_list.append(difficulty)
            if float(rating) in ratings_count or difficulty in difficulty_count:
                print(True)
                ratings_count[float(rating)] +=1
                difficulty_count[float(difficulty)] +=1
            else:
                print(True)
                ratings_count[float(rating)] = 1
                difficulty_count[float(difficulty)] =1
            print(difficulty_count)
            average_rating += float(rating)
            average_difficulty += float(difficulty)
            index +=1
else: #for filtering out classes
    course, filtered = input("enter the course number(s) of the classes that you would NOT like to see. Press enter on blank when you're complete: "), []
    while course:
        filtered.append(course.upper())
        course = input("enter the course number(s) of the classes that you would NOT like to see. Press enter on blank when you're complete: " )
    for name, review, rating, difficulty in zip(names, reviews, ratings, difficulties):
        print(name)
        if not any([name in course for course in filtered]):# i.e. class should NOT be CS61A filtered
            print(review, rating, difficulty, '\n')
            ratings_list.append(rating)
            difficulties_list.append(difficulty)
            if float(rating) in ratings_count or difficulty in difficulty_count:
                ratings_count[float(rating)] +=1
                difficulty_count[float(difficulty)] +=1
            else:
                ratings_count[float(rating)] = 1
                difficulty_count[float(difficulty)] =1
            average_rating += float(rating)
            average_difficulty += float(difficulty)
            index +=1    
try:
    print("Average rating under filters: {} \nAverage difficulty under filters: {}".format(average_rating/index, average_difficulty/index))
    print("Max/min rating under filters: {}, {} \nMax/min difficulty under filters: {}, {}".format(max(ratings_list), min(ratings_list), max(difficulties_list), min(difficulties_list)))
except ZeroDivisionError:
    pass

CS61A
At the moment that I'm writing this, this man's rating is a 4.7. That is criminally low: DeNero is an all-time professor who should be a straight 5. 5.0 4.0 

True
{1: 0, 2: 0, 3: 0, 4: 1, 5: 0}
61A
John DeNero made my CS 61A a lot more enjoyable. He gives amazing lectures and the tests are not hard as long as you study enough. 5.0 1.0 

True
{1: 1, 2: 0, 3: 0, 4: 1, 5: 0}
CS61A
Professor DeNero did a fantastic job with his class, especially given the transition to online learning during the Covid-19 pandemic. He is a great teacher who cares about how you're doing in his class and has many options to help struggling students. Also, his CS61A class is very useful and a great introduction to computer programming. 5.0 3.0 

True
{1: 1, 2: 0, 3: 1, 4: 1, 5: 0}
CS61A
I was very excited for this class however I was very disappointed. Lectures, labs, homeworks and even projects were way easier than midterms/final. There was a gap in difficulty which I didnt know how to cover. These test

In [64]:
#more data
#here i could put highest and lowest rating/review associated with it, average rating for specific classes
print(sum([val for val in ratings_count.values()]))
print(sum(difficulty_count.values()))
ratings_count, difficulty_count


110
58


({1: 3, 2: 2, 3: 5, 4: 6, 5: 85, 4.5: 7, 3.5: 1, 2.5: 1},
 {1: 9, 2: 9, 3: 22, 4: 15, 5: 3})

In [46]:
len(names), len(reviews), len(ratings), len(difficulties)

(118, 118, 118, 118)

In [ ]:
css_soup = bs('<p id="body"></p>')
css_soup.p['id']

In [ ]:
def filter_by_grades(thing):
    return any([s.lower() in ["a+", "b-","b+" ] for s in thing.split()]) 
letterz,gradesss = professor_soup.find_all(string = filter_by_grades), []
for letter in letterz:
    gradesss.append(grade_to_number(letter))
gradesss, average_grade(gradesss)